# 2. KL divergence and Maximum Likelihood

The Kullback-Leibler (KL) divergence is a measure of how much one probability distribution is different from a second one. It is a concept that originated in Information Theory, but has made its way into several other fields, including Statistics, Machine Learning, Information Geometry, and many more. In Machine Learning, the KL divergence plays a crucial role, connecting various concepts that might otherwise seem unrelated.

In this problem, we will introduce KL divergence over discrete distributions, practice some simple manipulations, and see its connection to Maximum Likelihood Estimation.

The KL divergence between two discrete-valued distributions $P(X),Q(X)$ over the outcome space $\mathcal{X}$ is defined as follows:

\begin{align*}
D_{KL}(P\|Q) = \sum_{x\in\mathcal{X}}P(x)\log\frac{P(x)}{Q(x)}.
\end{align*}

For notational convenience, we assume $P(x)>0, \forall x$ (One other standard thing to do is to adopt the convention that $0\log0 = 0$). Sometimes, we also write the KL divergence more explicitly as
$D_{KL}(P\|Q) = D_{KL}(P(X)\|Q(X)).$

_Background on Information Theory_

Before we dive deeper, we give a brief (optional) Information Theoretic background on KL divergence. While this introduction is not necessary to answer the assignment question, it may help you better understand and appreciate why we study KL divergence, and how Information Theory can be relevant to Machine Learning.

We start with the entropy $H(P)$ of a probability distribution $P(X)$, which is defined as
\begin{align*}
H(P) = -\sum_{x\in\mathcal{X}}P(x)\log P(x).
\end{align*}

Intuitively, entropy measures how dispersed a probability distribution is. For example, a uniform distribution is considered to have very high entropy (i.e. a lot of uncertainty), whereas a distribution that assigns all its mass on a single point is considered to have zero entropy (i.e. no uncertainty). Notably, it can be shown that among continuous distributions over $\mathbb{R}$, the Gaussian distribution $\mathcal{N}(\mu,\sigma^2)$ has the highest entropy (highest uncertainty) among all possible distributions that have the given mean $\mu$ and variance $\sigma^2$.

To further solidify our intuition, we present motivation from communication theory. Suppose we want to communicate from a source to a destination, and our messages are always (a sequence of) discrete symbols over space $\mathcal{X}$ (for example, $\mathcal{X}$ could be letters $\{a,b,\ldots,z\}$). We want to construct an encoding scheme for our symbols in the form of sequences of binary bits that are transmitted over the channel. Further, suppose that in the long run the frequency of occurrence of symbols follow a probability distribution $P(X)$. This means, in the long run, the fraction of times the symbol $x$ gets transmitted is $P(x)$.

A common desire is to construct an encoding scheme such that the average number of bits per symbol transmitted remains as small as possible. Intuitively, this means we want very frequent symbols to be assigned to a bit pattern having a small number of bits. Likewise, because we are interested in reducing the average number of bits per symbol in the long term, it is tolerable for infrequent words to be assigned to bit patterns having a large number of bits, since their low frequency has little effect on the long term average. The encoding scheme can be as complex as we desire, for example, a single bit could possibly represent a long sequence of multiple symbols (if that specific pattern of symbols is very common). The entropy of a probability distribution $P(X)$ is its optimal bit rate, i.e., the lowest average bits per message that can possibly be achieved if the symbols $x\in\mathcal{X}$ occur according to $P(X)$. It does not specifically tell us how to construct that optimal encoding scheme. It only tells us that no encoding can possibly give us a lower long term bits per message than $H(P)$.

To see a concrete example, suppose our messages have a vocabulary of $K=32$ symbols, and each symbol has an equal probability of transmission in the long term (i.e, uniform probability distribution). An encoding scheme that would work well for this scenario would be to have $\log_2 K$ bits per symbol, and assign each symbol some unique combination of the $\log_2 K$ bits. In fact, it turns out that this is the most efficient encoding one can come up with for the uniform distribution scenario.

It may have occurred to you by now that the long term average number of bits per message depends only on the frequency of occurrence of symbols. The encoding scheme of scenario A can in theory be reused in scenario B with a different set of symbols (assume equal vocabulary size for simplicity), with the same long term efficiency, as long as the symbols of scenario B follow the same probability distribution as the symbols of scenario A. It might also have occured to you, that reusing the encoding scheme designed to be optimal for scenario A, for messages in scenario B having a _different probability_ of symbols, will always be suboptimal for scenario B. To be clear, we do not need know _what_ the specific optimal schemes are in either scenarios. As long as we know the distributions of their symbols, we can say that the optimal scheme designed for scenario A will be suboptimal for scenario B if the distributions are different.

Concretely, if we reuse the optimal scheme designed for a scenario having symbol distribution $Q(X)$, into a scenario that has symbol distribution $P(X)$, the long term average number of bits per symbol achieved is called the cross entropy, denoted by $H(P,Q)$):

\begin{align*}
H(P,Q) = -\sum_{x\in\mathcal{X}}P(x)\log Q(x).
\end{align*}

*If P and Q are densities for continuous-valued random variables, then the sum is replaced by an integral, and everything stated in this problem works fine as well. But for the sake of simplicity, in this problem we’ll just work with this form of KL divergence for probability mass functions/discrete-valued distributions.*


To recap, the entropy $P(X)$ is the best possible long term average bits per message (optimal) that can be achived under a symbol distribution $P(X)$ by using an encoding scheme (possibly unknown)specifically designed for $P(X)$. The cross entropy $H(P,Q)$ is the long term average bits per message (suboptimal) that results under a symbol distribution $P(X)$, by reusing an encoding scheme (possibly unknown) designed to be optimal for a scenario with symbol distribution $Q(X)$.

Now, KL divergence is the penalty we pay, as measured in average number of bits, for using the optimal scheme for $Q(X)$, under the scenario where symbols are actually distributed as $P(X)$. It is straightforward to see this

\begin{align*}
D_{KL}(P\|Q) 
& = \sum_{x\in\mathcal{X}}P(x)\log\frac{P(x)}{Q(x)}\\
& = \sum_{x\in\mathcal{X}}P(x)\log Q(x) - \sum_{x\in\mathcal{X}}P(x)\log P(x)\\
& = H(P,Q) - H(P). \quad {\rm (difference\ in\ average\ number\ of\ bits.)}
\end{align*}

If the cross entropy between $P$ and $Q$ is zero (and hence $D_{KL}(P\|Q)=0$) then it necessarily means $P=Q$. In Machine Learning, it is a common task to find a distribution $Q$ that is “close” to another distribution $P$. To achieve this, we use $D_{KL}(P\|Q)$ to be the loss function to be optimized. As we will see in this question below, Maximum Likelihood Estimation, which is a commonly used optimization objective, turns out to be equivalent minimizing KL divergence between the training data (i.e. the empirical distribution over the data) and the model.

Now, we get back to showing some simple properties of KL divergence.


### (a) [5 points] Nonnegativity. 
Prove the following: 
\begin{align*}
\forall P,Q &\quad D_{KL}(P\|Q)\geq 0
\end{align*}
and
\begin{align*}
D_{KL}(P\|Q)= 0 &\ { \rm if\ and\ only\ if\ } P=Q.
\end{align*}

**Hint**: You may use the following result, called **Jensen's inequality**. If $f$ is a convex function, and $X$ is a random variable, then $E(f[X]) \geq f(E[X])$. Moreover, if f is strictly convex ($f$ is convex if its Hessian satisfies $H\geq 0$; it is strictly convex if $H>0$; for instance $f(x) = −\log x$ is strictly convex), then $E(f[X]) = f(E[X])$ implies that $X=E(X)$ with probability $1$; i.e., $X$ is actually a constant.

### Answer:

Notice that
\begin{align*}
D_{KL}(P\|Q) 
& = E_{x\sim P}\left(-\log\frac{Q(x)}{P(x)}\right)\\
& \geq -\log\left(E_{x\sim P}\big(\frac{Q(x)}{P(x)}\big)\right)\\
& = -\log(1) = 0
\end{align*}
which proves the first part of the question.
If $D_{KL}(P\|Q)= 0$, then 
\begin{align*}
E_{x\sim P}\left(-\log\frac{Q(x)}{P(x)}\right) = -\log\left(E_{x\sim P}\big(\frac{Q(x)}{P(x)}\big)\right).
\end{align*}
Using the fact that $f(x) =-\log x$ is strictly convex along with Jensen's inequality, $\frac{Q(x)}{P(x)} = E_{x\sim P}(\frac{Q(x)}{P(x)}) = 1$, as desired. The reverse assertion is trivial. 

---

### (b) [5 points] Chain rule for KL divergence. 

The KL divergence between $2$ conditional distributions $P(X|Y), Q(X|Y)$ is defined as follows:
\begin{align*}
D_{KL}(P(X|Y)\|Q(X|Y)) = -\sum_{y}P(y)\left(\sum_{x}P(x|y)\log \frac{P(x|y)}{Q(x|y)}\right).
\end{align*}
This can be thought of as the expected KL divergence between the corresponding conditional distributions on $x$ (that is, between $P(X|Y=y)$ and $Q(X|Y=y)$), where the expectation is taken over the random $y$.
Prove the following chain rule for KL divergence:
\begin{align*}
D_{KL}(P(X,Y)\|Q(X,Y)) = D_{KL}(P(X)\|Q(X)) + D_{KL}(P(Y\|X)\|Q(Y\|X)).
\end{align*}

### Answer:

\begin{align*}
D_{KL}(P(X,Y)\|Q(X,Y))
& = E_{x,y\sim P(x,y)}\Big(-\log\frac{P(x,y)}{Q(x,y)}\Big)\\
& = E_{x,y\sim P(x,y)}\Big(-\log\frac{P(x)P(y|x)}{Q(x)Q(y|x)}\Big)\\
& = E_{x,y\sim P(x,y)}\Big(-\log\frac{P(x)}{Q(x)}\Big) + E_{x,y\sim P(x,y)}\Big(-\log\frac{P(y|x)}{Q(y|x)}\Big)\\
& = E_{x\sim P(x)}\Big(-\log\frac{P(x)}{Q(x)}\Big) + D_{KL}(P(Y|X)\|Q(Y|X))\\
& = D_{KL}(P(X)\|Q(X)) + D_{KL}(P(Y|X)\|Q(Y|X))
\end{align*}

---

### (c) [5 points] KL and maximum likelihood. 

Consider a density estimation problem, and suppose we are given a training set $\{x^{(i)}; i=1,\ldots,m\}$. Let the empirical distribution be $\hat{P}(x) = \frac{1}{m}\sum_{i=1}^m1_{\{x^{(i)}=x\}}$. ($\hat{P}$ is just the uniform distribution over the training set; i.e., sampling from the empirical distribution is the same as picking a random example from the training set.)

Suppose we have some family of distributions $P_\theta$ parameterized by $\theta$. (If you like, think of $P_\theta(x)$ as an alternative notation for $P(x;\theta)$.) Prove that finding the maximum likelihood estimate for the parameter $\theta$ is equivalent to finding $P_\theta$ with minimal KL divergence from $\hat{P}$. I.e. prove:

\begin{align*}
 \arg\min_{\theta} D_{KL}(\hat{P}\|P_\theta) = \arg\max_{\theta}\sum_{i=1}^m \log P_\theta(x^{(i)}).
\end{align*}

### Answer:

\begin{align*}
\arg\min_{\theta} D_{KL}(\hat{P}\|P_\theta) 
& = \arg\min_{\theta} \left(H(\hat{P},P_\theta)-H(\hat{P}) \right)\\
& = \arg\min_{\theta} H(\hat{P},P_\theta)\\
& = \arg\min_{\theta} -\sum_{x\in \mathcal{X}} \hat{P}(x^{(i)})\log P_\theta(x^{(i)})\\
& = \arg\max_{\theta} \frac{1}{m}\sum_{i=1}^m \log P_\theta(x^{(i)}).
\end{align*}

---

**Remark.** Consider the relationship between parts (b-c) and multi-variate Bernoulli Naive Bayes parameter estimation. In the Naive Bayes model we assumed $P_\theta$ is of the following form: 
$P_\theta(x,y)=P(y)\prod_{i=1}^np(x_i\|y)$
By the chain rule for KL divergence, we therefore have:

\begin{align*}
D_{KL}\big(\hat{P}\|P_\theta\big) = D_{KL}\big(\hat{P}(y)\|P(y)\big) +\sum_{i=1}^n D_{KL}\big(\hat{P}(x_i|y)\|P(x_i|y)\big).
\end{align*}



This shows that finding the maximum likelihood/minimum KL-divergence estimate of the parameters decomposes into $2n+1$ independent optimization problems: One for the class priors $p(y)$, and one for each of the conditional distributions $p(x_i|y)$ for each feature $x_i$ given each of the two possible labels for $y$. Specifically, finding the maximum likelihood estimates for each of these problems individually results in also maximizing the likelihood of the joint distribution. (If you know what Bayesian networks are, a similar remark applies to parameter estimation for them.)